# Rascunho contagem de veículos
Para a câmera dos primeiros 23 minutos - https://www.youtube.com/watch?v=3uwRHtiyMnI

Este Notebook contem o código que criei como "rascunho" para entender como resolver o problema e, ao mesmo tempo, uma possível solução que posteriormente foi aprimorada na versão seguinte.

In [1]:
import cv2
from ultralytics import YOLO

O modelo pré-disponível do YOLO já possui as classes que preciso detectar, então vou aproveitar o modelo para o projeto. Somente para garantir um resultado melhor, irei utilizar a versão mais pesada que é o "X", neste momento não estou pensando em otimização, somente em validar a possibilidade de criar o sistema e criar as outras etapas que são a contagem em si (não quero me preocupar agora em anotar e treinar modelos).

In [2]:
model = YOLO('yolov8x')

In [3]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

Para facilitar minha vida, fiz a detecção de frames no vídeo até encontrar algum objeto para utilizar nas próximas etapas.

In [4]:
results_video = model.track(source='camera.mp4',  # image or video; single value or a list; URL, PIL (RGB), CV2 (BGR), ...
                            conf=0.25,
                            iou=0.7,
                            imgsz=640,
                            classes=[2, 3, 5, 7],
                            show=False,
                            save=True,
                            save_txt=False,  # Save bbox coordination
                            save_conf=False,  # save_txt must be True
                            save_crop=False,
                            verbose=False,
                            stream=True  # Do inference now (False) or after (True)
                           )
for result in results_video:
    if len(result.boxes.cls):
        break

In [5]:
result.boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 7., 2., 2., 2.])
conf: tensor([0.8638, 0.8509, 0.7668, 0.7581, 0.7230, 0.7204, 0.6943, 0.6914, 0.6080, 0.5529, 0.4777, 0.4333, 0.3480, 0.2592, 0.2581])
data: tensor([[9.5859e+02, 5.7417e+02, 1.0301e+03, 6.2251e+02, 1.0000e+00, 8.6379e-01, 2.0000e+00],
        [4.4731e+02, 5.8712e+02, 5.1338e+02, 6.4473e+02, 2.0000e+00, 8.5087e-01, 2.0000e+00],
        [4.8163e+02, 4.7221e+02, 5.2497e+02, 5.1029e+02, 3.0000e+00, 7.6677e-01, 2.0000e+00],
        [7.1962e+02, 4.0680e+02, 7.4181e+02, 4.2482e+02, 4.0000e+00, 7.5809e-01, 2.0000e+00],
        [5.1092e+02, 4.3721e+02, 5.4516e+02, 4.7007e+02, 5.0000e+00, 7.2302e-01, 2.0000e+00],
        [5.7998e+02, 4.1678e+02, 6.0729e+02, 4.4239e+02, 6.0000e+00, 7.2041e-01, 2.0000e+00],
        [5.4522e+02, 4.1599e+02, 5.6849e+02, 4.3348e+02, 7.0000e+00, 6.9435e-01, 2.0000e+00],
        [5.2747e+02, 4.0326e+02, 5.4749e+02, 4.1909e+02, 8.0000e+00, 

In [6]:
cv2.imwrite('img_original.png', result.orig_img)

True

Neste caso decidi utilizar o centro de cada box para usar como referência de onde o objeto se encontra.

In [7]:
center_x = int((result.boxes.xyxy[0][0] + result.boxes.xyxy[0][2]) / 2)
center_y = int((result.boxes.xyxy[0][1] + result.boxes.xyxy[0][3]) / 2)
center_x, center_y

(994, 598)

Identificando as melhores posições e coordenadas para botar as linhas de contagem e para criar o painel de contagem. Para facilitar estou desenhando na imagem (mais fácil de debugar), mas em si, será utilizado somente as coordenadas.

In [8]:
img = cv2.imread('img_original.png')
cv2.line(img, (649, 590), (180, 590), (0, 0, 255), 3)  # Left
cv2.line(img, (650, 531), (1023, 531), (255, 0, 0), 3)  # Right
cv2.line(img, (650, 1280), (650, 0), (255, 0, 255), 3)  # Division
cv2.circle(img, (994,597), color=(255, 255, 0), radius=1, thickness=10)

# White board (results)
cv2.rectangle(img, (10, 10), (230, 95), (255, 255, 255), thickness=-1)
cv2.rectangle(img, (10, 10), (230, 95), (0, 0, 0), thickness=2)
cv2.putText(img, f'Left: 0', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
cv2.putText(img, f'Right: 0', (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

cv2.imwrite('img_lines.png', img)

True

Código inicial da contagem. Simplifiquei, fazer uma linha de corte vertical separando as pistas, possibilitando reconhecer se o objeto está na esquerda ou direita para saber qual linha de contagem irei utilizar. Também adicionei um tratamento para evitar contar o mesmo objeto mais de uma vez e um buffer, o qual o tracker ID precisa ficar sem ser detectado por um tempo até considerar que não está mais presente na imagem (isso, pois o objeto pode ficar sem ser detectar por alguma falha ou obstrução por um curto período... algo normal).

In [9]:
counter = {'left': 0, 'right': 0}
detections = {}
side = 'left' if center_x < 650 else 'right'
if side == 'left':
    position_y = 'up' if center_y < 590 else 'down'
else:
    position_y = 'up' if center_y < 531 else 'down'

track_id = int(result.boxes.id[0])
# Track_id already detected
if track_id in detections:
    detections[track_id]['count_not_detected'] = 0
    if detections[track_id]['position_y'] != position_y:
        # Only consider those who crossed the line at the correct position_y, otherwise, treat it as a mistake and ignore it
        if side == 'left' and position_y == 'down' or side == 'right' and position_y == 'up':
            counter[side] += 1
            detections[track_id]['position_y'] = position_y
# New track_id
else:
    detections[track_id] = {'count_not_detected': 0, 'side': side, 'position_y': position_y}

# Remove any detections that no longer exist for N consecutive frames
for key in list(detections.keys()):
    detections[key]['count_not_detected'] += 1
    if detections[key]['count_not_detected'] >= 20:
        del detections[key]

In [10]:
detections

{1: {'count_not_detected': 1, 'side': 'right', 'position_y': 'down'}}